## A standard suite of tools to work with $SPRUCE$ measurements

- let's get some satellite imagery figured out with `rasterio`
- `astropy` will give appropriate motions of the sun; use the clinometer to get altitude

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import datetime as dt
from scipy import signal, interpolate
import os
import struct
import wave
import rasterio
import rasterio.plot
import rasterio.mask
import fiona
from shapely.geometry import box, LineString, MultiPoint, Polygon
import geopandas as gpd
import pysolar
import datetime as dt
import pytz

import sys
sys.path.append('/home/dax/PythonScripts/3_GITHUB/iyore')
import iyore

%matplotlib inline

def parse_SPRUCE(meas, start_time, tree):
    
    '''
    Load a raw .CSV file
    '''
    
    SPRUCE = pd.read_csv(meas, header=None) # read in the raw .CSV file
    SPRUCE.columns = ["ch1", "ch2", "ch3", "loop length (ms)"] # give the columns meaningful names
    SPRUCE = SPRUCE.replace(' NULL', np.NaN) # convert 'NULL' strings to numeric NaN
    SPRUCE["loop length (ms)"] = SPRUCE["loop length (ms)"].str[:-1] # strip trailing semicolon
    SPRUCE = SPRUCE.astype('float') # cooerce to floating point numbers

    # a meaningful datetime index
    SPRUCE.index = start_time + (SPRUCE["loop length (ms)"].cumsum()/1000).apply(lambda t: dt.timedelta(seconds=t))
    
    SPRUCE["tree"] = tree
    
    SPRUCE.name = tree

    return SPRUCE

def signal_to_wav(signal, fname, Fs):
    
    """Convert a numpy array into a wav file.

     Args
     ----
     signal : 1-D numpy array
         An array containing the audio signal.
     fname : str
         Name of the audio file where the signal will be saved.
     Fs: int
        Sampling rate of the signal.

    """
    data = struct.pack('<' + ('h'*len(signal)), *signal)
    wav_file = wave.open(fname, 'wb')
    wav_file.setnchannels(1)
    wav_file.setsampwidth(2)
    wav_file.setframerate(Fs)
    wav_file.writeframes(data)
    wav_file.close()
    
    print("complete.")
    
# audio-processing
import pydub
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
from pydub.utils import which
AudioSegment.converter = which("ffmpeg")
import crepe # pitch-tracking

# geo-processing
from shapely.geometry import Polygon, asPolygon, asMultiPoint, asPoint
from shapely.affinity import rotate
from shapely.ops import nearest_points

# core + technical
import subprocess 
from sklearn.decomposition import PCA
import os
import datetime as dt
import shutil
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import sys
import numpy as np
from numpy import inf
import pandas as pd




def match_target_amplitude(sound, target_dBFS):
    
    change_in_dBFS = target_dBFS - sound.dBFS
    
    return sound.apply_gain(change_in_dBFS)

def combine_wav_files(in_path, out_path, out_name):
    
    # routing
    in_path_wav = os.path.join(in_path, "*.wav")
    out_path_name = os.path.join(os.path.join(in_path, out_path), out_name + ".wav")
    
    # input
    paths = glob.glob(in_path_wav)

    stitch = AudioSegment.empty()
    for file in paths:

        sound_file = AudioSegment.from_wav(file)

        stitch+=sound_file
    
    # output
    stitch.export(out_path_name, format="wav")

    
def audio_chunker(audio_path, silence_thresh=-36.0, min_silence=1, len_thresh=200, pad=300):
    
    '''
    Wrapper to load MP3 or WAV files,
    then use `pydub.silence.split_on_silence` to separate 
    
    Inputs
    ------
    audio_path: (str, path) the file you would like to process - can be MP3 or WAV
    
    silence_thresh: (float) an amplitude threshold referenced to decibels full-scale (dBFS)
    
    min_silence: (float) "(in ms) minimum length of a silence to be used for
        a split. default: 1000ms"
        
    len_thresh: (float) ""(in ms or True/False) leave some silence at the beginning
        and end of the chunks. Keeps the sound from sounding like it
        is abruptly cut off.
        When the length of the silence is less than the keep_silence duration
        it is split evenly between the preceding and following non-silent
        segments.
        If True is specified, all the silence is kept, if False none is kept.
        default: 100ms"
        
    pad: (float) 
    
    
    Returns
    -------
    audio_chunks: (list) of `pydub.AudioSegment` objects representing periods of "non-silence"
    
    '''
    
    import pydub
    from pydub import AudioSegment
    from pydub.silence import split_on_silence
    from pydub.utils import mediainfo
    from pydub.playback import play
    from pydub.utils import which

    AudioSegment.converter = which("ffmpeg")
    
    # load the audio with `pydub`
    if(audio_path[-3:] == "wav"):
        sound_file = AudioSegment.from_wav(audio_path)

    elif((audio_path[-3:] == "MP3")|(audio_path[-3:] == "mp3")):
        sound_file = AudioSegment.from_mp3(audio_path)

    else:
        print("there's a pydub generic 'file' version, too, you know...")


    # grab file info while we're at it
    info = mediainfo(audio_path)

    # break it into chunks - the thresholds here might eventually benefit 
    # from some kind of NVSPL preview, I think
    audio_chunks = split_on_silence(sound_file, 
                                    min_silence_len=min_silence, 
                                    silence_thresh=silence_thresh,
                                    keep_silence=pad)
    
    return audio_chunks


def PCA_audio_sampler_prime(audio_path, fft_size=1024, silence_thresh=-36, min_silence=1, len_thresh=20,  
                            pad=300, target_amplitude=-12, compute_pitch=True, verbose=False):
    
    '''
    Making a lathe on which to turn audio.
    The premise here is to divide up the waveform (more or less dynamic) into chunks,
    perform a re-ordination and disperal routine [that is, Principle Component Analysis, PCA]
    and then progress through the dispersed samples using a rotor - however complex
    
    PCA - 
    
    
    '''
    
    audio_chunks = audio_chunker(audio_path, 
                                 silence_thresh=silence_thresh, 
                                 min_silence=min_silence, 
                                 len_thresh=len_thresh, 
                                 pad=pad)
    
    print("Settings produced", len(audio_chunks), "audio chunks")
    
    # grab file info for subsequent computations
    info = mediainfo(audio_path)
    
    # =================================================================
    
    # these are our returns
    normalized_data = []
    indices_out = []
    clip_pitch = []
    chunks_out = []
    
    for i, chunk in enumerate(audio_chunks):
        
        # amplify the output level to some new value
        chunk_normalized = match_target_amplitude(chunk, target_amplitude)
        chunks_out.append(chunk_normalized)

        # convert the `pydub` object to `numpy`
        samples = chunk_normalized.get_array_of_samples()
        samples = np.array(samples)

        # if stereo, use the L channel only
        if(len(samples.shape) > 1):
            samples = samples[:, 0]

        rate = int(info['sample_rate'])
        #rate = 44100

        # perform a short-time Fourier transform on the data
        frequencies, times, Zxx = signal.stft(samples, 
                                              rate, 
                                              nfft=fft_size)
    
        if(verbose):
            print(Zxx.shape, times.shape)

        if(i%100 == 0):
            print("chunk ", i+1)

        # create the spectrogram by dropping the imaginary (phase) component of the signal
        spectrogram = np.log(np.abs(Zxx))

        if((times.shape[0] > len_thresh)):
        #if((times.shape[0] > len_thresh)&(times.shape[0] < 80)):

            if(verbose):

                plt.figure(figsize=(3, 3))
                plt.pcolormesh(times, frequencies, spectrogram, cmap='magma')
                plt.title("Chunk "+str(i), loc="left")
                plt.show()
                plt.close()

            if(compute_pitch):

                # perform pitch detection
                time, frequency, confidence, activation = crepe.predict(samples, rate, viterbi=False, 
                                                                        step_size=len(samples), verbose=False)

                # if there is reasonable confidence, add it to the attributes
                if(confidence > 0.1):
                    clip_pitch.append(frequency[0])

                else:
                    clip_pitch.append(np.nan)

            else:
                clip_pitch.append(np.nan)

            # then flatten the spectrogram into a vector for PCA
            norm = spectrogram.flatten()/spectrogram.flatten().max()
            normalized_data.append(norm)
            indices_out.append(i)
        
    return chunks_out, normalized_data, indices_out, clip_pitch

def polygon_coords(center, radius, n):
    
    # set up each coordinate
    xs = center[0] + radius * np.cos(np.pi/n * (1 + 2 * np.arange(0, n)))
    ys = center[1] + radius * np.sin(np.pi/n * (1 + 2 * np.arange(0, n)))
    
    coords = np.array([xs, ys]).T
    
    ngon = asPolygon(coords)
    
    return ngon

def gold(value, operator="+", repeats=1):
    
    '''
    Golden ratio calculator; T A P E    M A T H
    
    operator
        "+"  scale up
        "-"  scale down
        
    repeats:  optional
        otherwise, int
    
    '''
    
    big = 1+np.sqrt(5)
    small = 2
    
    
    def up(value):
        
        phi_multiply = small/big - small
        
        value *= -phi_multiply # 1.381966%
        return value
    
    def down(value):
        
        #phi_divide = (small - big)/big
        phi_divide = big/small - small
        value *= -phi_divide # 0.381966%
        return value
    
    def repeat(fn, n):
        
        if n == 1:
            return fn
        else:
            def new_fn(x):
                return fn(repeat(fn, n-1)(x))

            return new_fn
    

    if ((operator == "+")|(operator == 1)|(operator == True)):
        out = repeat(up, repeats)(value)     # scale up
        return out
        
    elif ((operator == "-")|(operator == -1)|(operator == False)):
        out = repeat(down, repeats)(value)      # scale down
        return out
    

### create a masked version of the Alaska 5-meter DEM for this project

In [ ]:
# with rasterio.open("/home/dax/Documents/SPATIAL/Datasets/USGS_AK5M_AK_IFSAR_2010_58.tif") as src:
    
#     # create a rectangular mask
#     mask = gpd.GeoDataFrame([], 
#                         geometry=[box(-148.984514, 63.38777, -148.948851, 63.399623)],
#                         crs="EPSG:4326")
    
#     mask = mask.to_crs(src.crs)
    
#     print(mask.geometry[0])
    
#     out_image, out_transform = rasterio.mask.mask(src, [mask.geometry[0]], crop=True)
#     out_meta = src.meta
    
# out_meta.update({"driver": "GTiff",
#                  "height": out_image.shape[1],
#                  "width": out_image.shape[2],
#                  "transform": out_transform})

# with rasterio.open("/home/dax/Documents/SPATIAL/Datasets/USGS_AK5M_AK_IFSAR_2010_58_Cantwell.tif", "w", **out_meta) as dest:
#     dest.write(out_image)
    

In [ ]:
# ras = rasterio.open("/home/dax/Documents/SPATIAL/Datasets/USGS_AK5M_AK_IFSAR_2010_58_Cantwell.tif")

# plt.figure(figsize=(10, 10))
# rasterio.plot.show(ras, cmap="terrain")
# plt.show()

## Load *every* Cantwell Rock measurement

In [2]:
SPRUCE = iyore.Dataset(r"/home/dax/Documents/ALBUM/SPRUCE/Field Measurements")

# join all spruce measurements together
S = pd.concat([parse_SPRUCE(e.path,
                        dt.datetime(int(e.year), 
                                    int(e.month), 
                                    int(e.day), 
                                    int(e.begin_hour), 
                                    int(e.begin_minute)),
                                    e.location + " " + e.num) for e in SPRUCE.meas(location="CantwellRock")])


# # S.index = np.arange(len(S))

# trim off the rather extreme bits
S.loc[:, "ch1":"ch3"] = S.loc[:, "ch1":"ch3"].mask(S.loc[:, "ch1":"ch3"].abs() > 0.2)

print("Spruce data loaded! Geoprocessing...")

S = S.loc["2022-08-16 12:00:00":, :]

tree_lat, tree_long = 63.397140, -148.973065

# defining the timezone
tz = pytz.timezone('US/Alaska')
        
irradiations = []
azimuths = []
for idx in S.index[::60*10]:
    
    print(idx)

    aware_idx = tz.localize(idx.to_pydatetime())
    alt = pysolar.solar.get_altitude(tree_lat, tree_long, aware_idx)
    azi = pysolar.solar.get_azimuth(tree_lat, tree_long, aware_idx)
    irr = pysolar.radiation.get_radiation_direct(aware_idx, alt)

    S.loc[idx, "Azi"] = azi
    S.loc[idx, "Alt"] = alt
    S.loc[idx, "Irr"] = irr
        
S = S.dropna()

print("Geoprocessing complete!")

Spruce data loaded! Geoprocessing...


/tmp/ipykernel_41521/3447503845.py:20: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  S = S.loc["2022-08-16 12:00:00":, :]


2022-08-16 20:37:00.235000


/home/dax/anaconda3/envs/geosound/lib/python3.9/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


2022-08-16 20:38:48.365000
2022-08-16 20:40:39.193000
2022-08-16 20:42:30.263000
2022-08-16 20:44:21.311000
2022-08-16 20:46:12.376000
2022-08-16 20:48:03.722000
2022-08-16 20:49:54.761000
2022-08-16 20:51:45.782000
2022-08-16 20:53:36.780000
2022-08-16 20:55:27.774000


...

2022-08-31 09:55:09.037000
2022-08-31 09:56:57.148000
2022-08-31 09:58:45.277000
2022-08-31 10:00:33.366000
2022-08-31 10:02:21.475000
2022-08-31 10:04:09.539000
2022-08-31 10:05:57.609000
2022-08-31 10:07:45.670000
2022-08-31 10:09:33.716000
2022-08-31 10:11:21.735000
2022-08-31 10:13:09.722000
2022-08-31 10:14:57.707000
2022-08-31 10:16:45.706000
2022-08-31 10:18:33.602000
2022-08-31 10:20:21.464000
2022-08-31 10:22:09.281000
2022-08-31 10:23:57.088000
2022-08-31 10:25:44.898000
2022-08-31 10:27:32.725000
2022-08-31 10:29:20.565000
2022-08-31 10:31:08.398000
2022-08-31 10:32:56.231000
2022-08-31 10:34:44.090000
2022-08-31 10:36:31.923000
2022-08-31 10:38:19.787000
2022-08-31 10:40:07.633000
2022-08-31 10:41:55.409000
2022-08-31 10:43:43.201000
2022-08-31 10:45:31.019000
2022-08-31 10:47:18.862000
2022-08-31 10:49:06.685000
2022-08-31 10:50:54.459000
2022-08-31 10:52:42.241000
2022-08-31 10:54:29.981000
2022-08-31 10:56:17.755000
2022-08-31 10:58:05.522000
2022-08-31 10:59:53.296000
2

2022-08-31 19:08:50.431000
2022-08-31 19:10:37.148000
2022-08-31 19:12:23.920000
2022-08-31 19:14:11.218000
2022-08-31 19:15:58.538000
2022-08-31 19:17:45.869000
2022-08-31 19:19:33.190000
2022-08-31 19:21:20.514000
2022-08-31 19:23:07.851000
2022-08-31 19:24:55.203000
2022-08-31 19:26:42.557000
2022-08-31 19:28:29.915000
2022-08-31 19:30:17.262000
2022-08-31 19:32:04.596000
2022-08-31 19:33:51.948000
2022-08-31 19:35:39.329000
2022-08-31 19:37:26.703000
2022-08-31 19:39:14.237000
2022-08-31 19:41:02.280000
2022-08-31 19:42:50.303000
2022-08-31 19:44:38.343000
2022-08-31 19:46:26.385000
2022-08-31 19:48:15.370000
2022-08-31 19:50:03.427000
2022-08-31 19:51:51.487000
2022-08-31 19:53:39.557000
2022-08-31 19:55:27.623000
2022-08-31 19:57:15.697000
2022-08-31 19:59:03.766000
2022-08-31 20:00:51.831000
2022-08-31 20:02:39.883000
2022-08-31 20:04:27.987000
2022-08-31 20:06:16.080000
2022-08-31 20:08:04.175000
2022-08-31 20:09:52.250000
2022-08-31 20:11:40.316000
2022-08-31 20:13:28.369000
2

2022-09-01 04:23:22.085000
2022-09-01 04:25:10.311000
2022-09-01 04:26:58.563000
2022-09-01 04:28:46.799000
2022-09-01 04:30:35.048000
2022-09-01 04:32:23.293000
2022-09-01 04:34:11.523000
2022-09-01 04:35:59.754000
2022-09-01 04:37:48.004000
2022-09-01 04:39:36.277000
2022-09-01 04:41:24.541000
2022-09-01 04:43:12.783000
2022-09-01 04:45:01.024000
2022-09-01 04:46:49.232000
2022-09-01 04:48:37.441000
2022-09-01 04:50:25.664000
2022-09-01 04:52:13.889000
2022-09-01 04:54:02.147000
2022-09-01 04:55:50.378000
2022-09-01 04:57:38.627000
2022-09-01 04:59:26.190000
2022-09-01 05:01:12.843000
2022-09-01 05:03:00.031000
2022-09-01 05:04:47.431000
2022-09-01 05:06:34.793000
2022-09-01 05:08:22.198000
2022-09-01 05:10:09.605000
2022-09-01 05:11:57.008000
2022-09-01 05:13:44.447000
2022-09-01 05:15:31.847000
2022-09-01 05:17:19.276000
2022-09-01 05:19:06.703000
2022-09-01 05:20:54.272000
2022-09-01 05:22:43.115000
2022-09-01 05:24:30.535000
2022-09-01 05:26:17.997000
2022-09-01 05:28:05.442000
2

2022-09-01 13:36:11.088000
2022-09-01 13:37:59.198000
2022-09-01 13:39:47.301000
2022-09-01 13:41:35.423000
2022-09-01 13:43:24.129000
2022-09-01 13:45:12.246000
2022-09-01 13:47:00.411000
2022-09-01 13:48:48.553000
2022-09-01 13:50:36.716000
2022-09-01 13:52:24.856000
2022-09-01 13:54:12.984000
2022-09-01 13:56:01.124000
2022-09-01 13:57:49.294000
2022-09-01 13:59:37.480000
2022-09-01 14:01:25.673000
2022-09-01 14:03:13.791000
2022-09-01 14:05:01.918000
2022-09-01 14:06:50.009000
2022-09-01 14:08:38.108000
2022-09-01 14:10:26.219000
2022-09-01 14:12:14.335000
2022-09-01 14:14:02.480000
2022-09-01 14:15:50.660000
2022-09-01 14:17:38.852000
2022-09-01 14:19:28.600000
2022-09-01 14:21:16.777000
2022-09-01 14:23:04.995000
2022-09-01 14:24:53.166000
2022-09-01 14:26:41.372000
2022-09-01 14:28:29.603000
2022-09-01 14:30:17.822000
2022-09-01 14:32:06.047000
2022-09-01 14:33:54.227000
2022-09-01 14:35:42.422000
2022-09-01 14:37:30.584000
2022-09-01 14:39:18.720000
2022-09-01 14:41:06.880000
2

2022-09-01 22:49:09.791000
2022-09-01 22:50:57.574000
2022-09-01 22:52:46.466000
2022-09-01 22:54:34.511000
2022-09-01 22:56:22.242000
2022-09-01 22:58:09.967000
2022-09-01 22:59:57.720000
2022-09-01 23:01:45.450000
2022-09-01 23:03:33.215000
2022-09-01 23:05:20.981000
2022-09-01 23:07:08.747000
2022-09-01 23:08:56.515000
2022-09-01 23:10:44.283000
2022-09-01 23:12:32.042000
2022-09-01 23:14:19.800000
2022-09-01 23:16:07.588000
2022-09-01 23:17:55.388000
2022-09-01 23:19:43.179000
2022-09-01 23:21:30.950000
2022-09-01 23:23:18.709000
2022-09-01 23:25:06.442000
2022-09-01 23:26:54.188000
2022-09-01 23:28:41.933000
2022-09-01 23:30:29.715000
2022-09-01 23:32:17.482000
2022-09-01 23:34:05.261000
2022-09-01 23:35:53.051000
2022-09-01 23:37:40.818000
2022-09-01 23:39:28.606000
2022-09-01 23:41:16.401000
2022-09-01 23:43:04.170000
2022-09-01 23:44:51.941000
2022-09-01 23:46:39.752000
2022-09-01 23:48:27.570000
2022-09-01 23:50:15.386000
2022-09-01 23:52:03.151000
2022-09-01 23:53:50.900000
2

In [3]:
S

,ch1,ch2,ch3,loop length (ms),tree,Azi,Alt,Irr
loop length (ms),,,,,,,,
2022-08-16 20:37:00.235,-0.0791,0.0421,0.0195,235.0,CantwellRock 001,284.248063,8.017793,260.290608
2022-08-16 20:38:48.365,-0.0861,0.0357,0.0021,185.0,CantwellRock 001,284.643880,7.824426,251.203372
2022-08-16 20:40:39.193,-0.0926,0.0350,-0.0157,184.0,CantwellRock 001,285.049567,7.626698,241.786783
2022-08-16 20:42:30.263,-0.0915,0.0351,-0.0157,184.0,CantwellRock 001,285.456145,7.429027,232.251603
2022-08-16 20:44:21.311,-0.0920,0.0361,-0.0178,185.0,CantwellRock 001,285.862657,7.231899,222.627516
...,...,...,...,...,...,...,...,...
2022-09-02 06:59:20.794,-0.1019,0.0505,0.0288,179.0,CantwellRock 001,74.007243,1.136513,0.060178
2022-09-02 07:01:08.648,-0.1032,0.0515,0.0292,179.0,CantwellRock 001,74.408531,1.309968,0.221248
2022-09-02 07:02:56.503,-0.1024,0.0522,0.0309,193.0,CantwellRock 001,74.809663,1.485292,0.605612


# $x_1 = V_1 \ cos(\phi_1), y_1 = V_1 \ sin(\phi_1), z_1=0$
# $x_2 = V_2 \ cos(\phi_2), y_2 = V_2 \ sin(\phi_2), z_2=0$
# $x_3 = V_3 \ cos(\phi_3), y_3 = V_3 \ sin(\phi_3), z_3=0$
# $x_4 = I \ cos(\phi_{sun}) sin(\alpha_{sun}), y_4 = I \ sin(\phi_{sun})sin(\alpha_{sun}), z_4=I cos(\theta_{sun})$

## see [ellipsoidal coordinates](https://en.wikipedia.org/wiki/Ellipsoidal_coordinates)

# $c_x = (\frac{V_1 cos(\phi_1) + V_2 cos(\phi_2) + V_3 cos(\phi_3) + I cos(\phi_{sun}) sin(\alpha_{sun})}{4})$
# $c_y = (\frac{V_1 sin(\phi_1) + V_2 sin(\phi_2) + V_3 sin(\phi_3) + I sin(\phi_{sun})sin(\alpha_{sun})}{4})$
# $c_z = (\frac{I cos(\alpha_{sun})}{4})$

In [4]:
phi = {"ch1":np.deg2rad(285), 
           "ch2":np.deg2rad(12), 
           "ch3":np.deg2rad(140)}

S = S.sort_index()

s_scale = 1e-4

c_x = (S["ch1"]*np.cos(phi["ch1"]) + S["ch2"]*np.cos(phi["ch2"]) +
       S["ch3"]*np.cos(phi["ch3"]) +
       s_scale*S["Irr"]*np.cos(np.deg2rad(S["Azi"]))*np.sin(np.deg2rad(S["Alt"])))/4

c_y = (S["ch1"]*np.sin(phi["ch1"]) + 
       S["ch2"]*np.sin(phi["ch2"]) +              
       S["ch3"]*np.sin(phi["ch3"]) + 
       s_scale*S["Irr"]*np.sin(np.deg2rad(S["Azi"]))*np.sin(np.deg2rad(S["Alt"])))/4

c_z = (np.cos(np.deg2rad(S["Alt"])))/4

S["cx"] = c_x
S["cy"] = c_y
S["cz"] = c_z

# find "zeros" 
# because Principal Components will likely be centered around (0, 0, 0)
S_x0 = (S.cx.max() - S.cx.min())/2
S_y0 = (S.cy.max() - S.cy.min())/2
S_z0 = (S.cz.max() - S.cz.min())/2

S_x0, S_y0, S_z0

# -------- DIAGNOSTIC PLOT -----------


%matplotlib qt

import matplotlib.dates as dates
plt_dates = dates.date2num(S.index.to_pydatetime())

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.plot(S.cx - S_x0, 
        S.cy - S_y0, 
        S.cz - S_z0, 
        color="k", ls="-", lw=0.5, ms=2, marker="o", alpha=0.1)

ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")

# ax.view_init(elev=45, azim=30)

plt.show()

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


## Part Two: PCA

In [36]:
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 15 SPRUCE samples.wav" # silence -60.0
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 27 SPRUCE woodyard hones.wav" # silence -55.0
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 27 SPRUCE small arms.wav" # silence -47.2
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 01 butt dome compressed clar.wav"
# audio_path = r"/home/dax/Documents/ALBUM/SPRUCE/Auditization/2022 10 15 SPRUCE drumsticks.wav"
# audio_path = r"/media/dax/4/DENAMTVI_2014.wav"
# audio_path = r"/media/dax/4/_KATMDUPO_2016.wav"
audio_path = r"/media/dax/4/DENAUTOK_2019.wav"

save_path = r"/home/dax/Documents/ALBUM/SPRUCE/Turning"

# <font color="salmon">$Tape Math$</font>

In [ ]:
gold(5, "+", 8)/1000

# stereo = interpolate.interp1d([0, 180], [-100, 100])
# stereo(12)



In [67]:
2048*2

4096

In [81]:
audio_chunks, normalized_data, indices_out, clip_pitch = PCA_audio_sampler_prime(audio_path, 
                                                                   fft_size=1024, 
                                                                   silence_thresh=-46.0, 
                                                                   min_silence=1, # int (ms), should be small at first 
                                                                                   # to let in samples; you can always turn it up
                                                                                 
                                                                   len_thresh=10.0, # we'd like relatively long samples (ex., ms); 
                                                                                  #  this will floor the PCA analysis
                                                                   
                                                                   pad=25,       # this pad should be large (ex. 500 ms), or 'True'
                                                                   target_amplitude=-25.0, # headroom (should be large!)
                                                                   compute_pitch=False, 
                                                                   verbose=False)


Settings produced 20956 audio chunks
chunk  1


/home/dax/anaconda3/envs/geosound/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:1961: UserWarning: nperseg = 256 is greater than input length  = 176, using nperseg = 176
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/dax/anaconda3/envs/geosound/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:1961: UserWarning: nperseg = 256 is greater than input length  = 178, using nperseg = 178
  warnings.warn('nperseg = {0:d} is greater than input length '


chunk  101
chunk  201
chunk  301
chunk  401
chunk  501
chunk  601
chunk  701
chunk  801
chunk  901
chunk  1001
chunk  1101
chunk  1201
chunk  1301
chunk  1401
chunk  1501
chunk  1601
chunk  1701
chunk  1801
chunk  1901
chunk  2001
chunk  2101
chunk  2201
chunk  2301
chunk  2401
chunk  2501
chunk  2601
chunk  2701
chunk  2801
chunk  2901
chunk  3001
chunk  3101
chunk  3201
chunk  3301
chunk  3401
chunk  3501
chunk  3601
chunk  3701
chunk  3801
chunk  3901
chunk  4001
chunk  4101
chunk  4201
chunk  4301
chunk  4401
chunk  4501
chunk  4601
chunk  4701
chunk  4801
chunk  4901
chunk  5001
chunk  5101
chunk  5201
chunk  5301
chunk  5401
chunk  5501
chunk  5601
chunk  5701
chunk  5801
chunk  5901
chunk  6001
chunk  6101
chunk  6201
chunk  6301
chunk  6401
chunk  6501
chunk  6601
chunk  6701
chunk  6801
chunk  6901
chunk  7001
chunk  7101
chunk  7201
chunk  7301
chunk  7401
chunk  7501
chunk  7601
chunk  7701
chunk  7801
chunk  7901
chunk  8001
chunk  8101
chunk  8201
chunk  8301
chunk  8401
c

### Principal Components Analysis

In [82]:
sample_lengths = np.array([len(n) for n in normalized_data])

# what percentage do we want to preserve?
preserve = 50 # % ####################################################### MAIN PARAMETER
p = 100 - preserve
shortest_vector = int(np.percentile(sample_lengths, p))

training_data_raw = []
for sample in normalized_data:
    if len(sample) > shortest_vector:
        training_data_raw.append(sample[:shortest_vector])
        
training_data = np.vstack(training_data_raw)
training_data[training_data == -inf] = 0

pca=PCA(n_components=3)
pca.fit(training_data)

print(pca.explained_variance_ratio_, pca.singular_values_)

projected = pca.fit_transform(training_data)

# ----------  PLOT THE RESULTS  ------------------

fig, ax = plt.subplots(1, 2, figsize=(9,4.8))
ax[0].scatter(projected[:, 0], projected[:, 1], s=1, color="k")
ax[0].set_xlabel('principle component 1', labelpad=15)
ax[0].set_ylabel('principle component 2', labelpad=10)
ax[1].scatter(projected[:, 1], projected[:, 2], s=1, color="k")
ax[1].set_xlabel('principle component 2', labelpad=15)
ax[1].set_ylabel('principle component 3', labelpad=10)

x, y = (0, 0)
ax[0].axhline(y, color="magenta")
ax[0].axvline(x, color="magenta")
ax[1].axhline(y, color="lime")
ax[1].axvline(x, color="lime")

ax[0].set_title("PCA on audio clips", loc="left", fontsize=14, y=1.02)
# plt.savefig(r"C:\Users\DBetchkal\Desktop\PCA_learning.png", dpi=100, bbox_inches="tight")

plt.tight_layout()
plt.show()

[0.25210756 0.06611659 0.0477641 ] [391.1923  200.33304 170.27393]


### Take $PC_1$, $PC_2$, $PC_3$ and map onto the range of $c_x$, $c_y$, $c_z$

In [83]:
# select only the 3D coordinates computed above
pathway = S.loc[:, ["cx", "cy", "cz"]].values

# find the ranges of all 3D coordinates
PC1_min, PC2_min, PC3_min = projected.min(axis=0)
PC1_max, PC2_max, PC3_max = projected.max(axis=0)
x_min, y_min, z_min = pathway.min(axis=0)
x_max, y_max, z_max = pathway.max(axis=0)

# set up interpolation functions
# for eventual sample selection
map_x = interpolate.interp1d([x_min, x_max],
                             [PC1_min, PC1_max])

map_y = interpolate.interp1d([y_min, y_max],
                             [PC2_min, PC2_max])

map_z = interpolate.interp1d([z_min, z_max],
                             [PC3_min, PC3_max])

how_far_apart = 2 # steps
audio_out = AudioSegment.empty()
for step in pathway[::how_far_apart, :]:
    
    
    
    selector = np.array([map_x(step[0]),
                         map_y(step[1]),
                         map_z(step[2])])
    
    dist_ind = np.sqrt(np.sum((projected-selector)**2, axis=1)) 

    audio_out = audio_out + audio_chunks[np.argmin(dist_ind)]
    

In [84]:
# get the datetimes as strings
startdt_out, enddt_out = [dt.datetime.strftime(t, "%Y%m%d_%H%M%S") for t in [S.index.min(), S.index.max()]]

# get the site
site = "CantwellRock 01"

# version
v = 9

filename = site + " begin " + startdt_out + " end " + enddt_out + " v" + str(v) + ".wav"
audio_out.export(save_path + os.sep + filename, format="wav")

audio_out

KeyboardInterrupt: 

### Wa

In [ ]:
def closest_point_and_distance(p, a, b):
    s = b - a
    w = p - a
    ps = np.dot(w, s)
    if ps <= 0:
        return a, np.linalg.norm(w)
    l2 = np.dot(s, s)
    if ps >= l2:
        closest = b
    else:
        closest = a + ps / l2 * s
    return closest, np.linalg.norm(p - closest)


point = [0, 0, 0]
seg = [[1, 1, 1], [1e3, 1e3, 1e3]] # the point of interest and a much larger value
closest_point_and_distance(point, np.array(seg[0]), np.array(seg[1]))

In [ ]:
def c(S):
    
    phi = {"ch1":np.deg2rad(285), 
           "ch2":np.deg2rad(12), 
           "ch3":np.deg2rad(140)}
    
    for idx, row in S.iterrows():
        
        c_x = 
        c_y = (S["ch1"]*np.sin(phi["ch1"]) + S["ch2"]*np.sin(phi["ch2"]) + S["ch2"]*np.sin(phi["ch2"]) + )/4

In [ ]:
# S = S.dropna()

dir_dict = {2:12, 3:140, 1:285}
color_dict = {1:"red", 2:"green", 3:"blue"}

fig, ax = plt.subplots(3, 1, figsize=(8, 5.5), sharex=True)

for channel in np.arange(3):

    ax[channel].plot(S["Azimuth"], S["ch"+str(channel+1)], ls="", marker="o", ms=1, color=color_dict[channel+1])
    ax[channel].axvline(dir_dict[channel+1], color="magenta", zorder=1)
#     ax[channel].set_ylim(S.loc[:, "ch1":"ch3"].min().min() - 0.05, 
#                          S.loc[:, "ch1":"ch3"].max().max() + 0.05)


plt.xlabel("Sun Azimuth (°)", labelpad=20, fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
wa

In [ ]:
wa = S.dropna()
channel = 1

%matplotlib qt

import matplotlib.dates as dates
plt_dates = dates.date2num(S.index.to_pydatetime())

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.plot(plt_dates, 1000*wa["ch"+str(channel)], wa["Irradiation"], 
        color="k", ls="", ms=1, marker="o", alpha=0.5)

ax.set_xlabel("Sun Azimuth (°)")
ax.set_ylabel("Voltage (mV)")
ax.set_zlabel("Solar Irradiation (W)")

# ax.view_init(elev=45, azim=30)

plt.show()

In [ ]:
print("interior angles", "\n\n", 
      "from 1 to 2:", str((360-285)+12)+"°\n",
      "from 2 to 3:", str(140-12)+"°\n",
      "from 3 to 1:", str(285-140)+"°\n")

print(87+128+145)

In [ ]:
# title = dt.datetime.strftime(S.index.min(), "%Y %m %d") + " " + "ALL"
# title = "ALL MEASUREMENTS TOGETHER"

S = S.loc[S["tree"] == 'CantwellRock 001']

title = "Cantwell Rock TOGETHER"
linesPerSave = 250

fig, ax = plt.subplots(2, 1, figsize=(14, 5), sharex=True)
ax[0].plot(S.index, 1000*S["ch1"], 
           color="red", ls="", marker="o", ms=1, label=dir_dict["ch1"])
ax[0].plot(S.index, 1000*S["ch2"], 
           color="green", ls="", marker="o", ms=1, label=dir_dict["ch2"])
ax[0].plot(S.index, 1000*S["ch3"], 
           color="blue", ls="", marker="o", ms=1, label=dir_dict["ch3"])
ax[0].set_title(title, fontsize=14, loc="left", y=1.05)

ax[0].set_ylabel("Voltage (mV)", labelpad=20, fontsize=12)
ax[0].xaxis.set_major_formatter(mdates.DateFormatter("%m %d\n%H:%M"))

ax[0].axhline(0, ls="--", color="gray", zorder=20)

ax[0].legend(bbox_to_anchor=(1.01, 0.5))
ax[0].set_ylim(-200, 200)

# time_series = time_series.loc[S.index.min():S.index.max()]
# ax[1].plot(time_series.index, time_series["t, 2m"], 
#            color="k", ls="-", label="temp C")

# # draw the 
# for i in np.arange(0, len(S), linesPerSave):
#     ax.axvline(S.iloc[i, :].name, color="red", ls="--", alpha=0.1, zorder=2)

# plt.savefig(r"/home/dax/Documents/ALBUM/Spruce/Field Testing" + os.sep + title + ".png",
#             dpi=150, bbox_inches="tight", facecolor="white")

# ax.set_xlim(S.index.min(), S.index.min()+dt.timedelta(minutes=20))


plt.show()

In [ ]:
help(dt.datetime.replace)

In [ ]:
S.index.max()

In [ ]:
# start_clip = "2022-08-17 00:00:00"
# plt.plot(S.loc[start_clip:, "ch1"], time_series[start_clip:])

S.index = S.index.to_series().apply(lambda t: t.replace(microsecond=0))

J = time_series.merge(S, how="inner", left_index=True, right_index=True)
J

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(1000*J["ch1"], 
         1000*J["t, 2m"], 
         ls="", ms=1, marker="o", alpha=0.2)
plt.show()

In [ ]:
start_clip = "2022-08-17 00:00:00"
plt.figure(figsize=(6, 6))
plt.plot(1000*S.loc[start_clip:, "ch1"], 
         1000*S.loc[start_clip:, "ch2"], 
         ls="", ms=1, marker="o", color="k", alpha=0.2)
plt.show()

In [ ]:
S["Hour"] = S.index.to_series().dt.hour

hourly = np.array([(hr, np.nanpercentile(group.loc[:, ["ch1", "ch2", "ch3"]], 50, axis=0)) for hr, group in S.groupby("Hour")])
hours, medians = hourly.T

channels = pd.DataFrame([list(p) for p in medians], columns=["ch1", "ch2", "ch3"])
plt.plot(channels.index.astype('int'), channels["ch1"], label="ch1", color="red")
plt.plot(channels.index.astype('int'), channels["ch2"], label="ch2", color="green")
plt.plot(channels.index.astype('int'), channels["ch3"], label="ch3", color="blue")
plt.show()

In [ ]:
np.nanpercentile(S.loc[:, ["ch1", "ch2", "ch3"]], 50, axis=0)

In [ ]:
step = 30 # mins
time_steps = np.arange(S.index.min(), S.index.max(), dt.timedelta(minutes=step))


for i, time_step in enumerate(time_steps):
    
    ts = pd.to_datetime(time_step)
    y_max = S.max()[:3].max()
#     y_max = 0.02
#     y_max = 0.1
    y_min = S.min()[:3].min()
#     y_min = 0.0
    
    fig, ax = plt.subplots(3, 1, figsize=(12, 3), sharex=True)
    ax[0].plot(S.loc[ts:ts+dt.timedelta(minutes=step), "ch1"], lw=0.5, color="red")
    ax[1].plot(S.loc[ts:ts+dt.timedelta(minutes=step), "ch2"], lw=0.5, color="green")
    ax[2].plot(S.loc[ts:ts+dt.timedelta(minutes=step), "ch3"], lw=0.5, color="blue")
    
    for a in ax:
        a.set_ylim([y_min, y_max])
        a.xaxis.set_major_formatter(mdates.DateFormatter("%m %d\n%H:%M:%S"))
        
    plt.tight_layout()
    plt.show()
    plt.close()

### A 3D plot

In [ ]:


    print(tree)

In [ ]:
# for creating a responsive plot
# %matplotlib widget

%matplotlib qt

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
step = 50

color_dict = {"BlueHome 001":"blue", "BlueHome 002":"blue", "BlueHome 003":"navy",
              "Brushkana 001":"orange", "Brushkana 003":"salmon",
              "CantwellRock 001":"k", "DenaliHQ 001":"green"}

for tree, obs in S.groupby("tree"):

    ax.plot(obs["ch1"][::step], obs["ch2"][::step], obs["ch3"][::step], 
            color=color_dict[tree], ls="", ms=1, marker="o", alpha=0.5)
ax.view_init(elev=45, azim=30)
plt.show()

In [ ]:
%matplotlib inline

rate = 44100
fft_size = 512

# perform a short-time Fourier transform on the data
frequencies, times, Zxx = signal.stft(S["ch2"], 
                                      rate, 
                                      nfft=fft_size)

spectrogram = np.log(np.abs(Zxx))

plt.figure(figsize=(10, 1))
plt.pcolormesh(times, frequencies, spectrogram, cmap='magma')
plt.title("Spectrogram Representation", loc="left")
plt.show()
plt.close()

In [ ]:
# if(save):
#     print(workingDir+os.sep+titleBar+".wav")

#     signal_to_wav((out_data).astype('int'), 
#                   workingDir+os.sep+titleBar+".wav", 
#                   playback_rate)

playback_rate = 11000
IPython.display.Audio(S["ch3"], rate=playback_rate)

# playback_rate = 10000

# for col in S:
#     if len(col) == 3:
        
#         out_filename = os.path.dirname(meas) + os.sep + os.path.basename(meas)[:-4] + "_" + col + "_audio_" + str(playback_rate) + "Hz.wav"
#         signal_to_wav((10000*(S[col]/S[col].max())).astype('int'),
#                       out_filename,
#                       playback_rate)
